In [1]:
%load_ext sql
%sql duckdb://

# Creating and Managing Indexes

- Indexes are used to improve the performance of queries by allowing the database to quickly locate the rows that match certain criteria.
- The `CREATE INDEX` statement is used to create an index on one or more columns of a table.
- The `CREATE UNIQUE INDEX` statement creates a unique index, which ensures that the indexed columns contain only unique values.  An __error will be raised__ if you violate the uniqueness.
- Indexes can be created on a single column or multiple columns.
- Constraints are used to enforce rules and relationships on the data in a table.
- The `ALTER TABLE` statement is used to add constraints to an existing table.
- The `PRIMARY KEY` constraint defines a column or a set of columns as the primary key for a table.
- The `UNIQUE` constraint ensures that the values in the specified column(s) are unique.
- The `FOREIGN KEY` constraint establishes a relationship between two tables by referencing the primary key of another table.
- Indexes and constraints can be dropped using the `DROP INDEX` and `ALTER TABLE DROP CONSTRAINT` statements, respectively.

Note: indices are __dynamic__ and result in __performance implications__ when adding/removing data in the future, but may improve query times.  Indicares are __transparent__ to the user once built.

## Declaring on Table Creation

In [7]:
%%sql

CREATE OR REPLACE TABLE employees (
    id INT PRIMARY KEY, -- id is indexed uniquely
    name VARCHAR(100),
    age INT UNIQUE, -- age is indexed uniquely
    salary DECIMAL(10, 2)
);

Running query in 'duckdb://'

Count


## Indexing Retroactively

In [8]:
%%sql

CREATE INDEX idx_name ON employees (name);
CREATE UNIQUE INDEX idx_id ON employees (id);
CREATE INDEX idx_age_salary ON employees (age, salary);
CREATE UNIQUE INDEX idx_name_age ON employees (name, age);

Running query in 'duckdb://'

Count


## Adding Constraints Retroactively

Note: this syntax is not supported by DuckDB/JupySQL and is just here for demonstration.

In [9]:
%%sql

ALTER TABLE employees ADD CONSTRAINT pk_id PRIMARY KEY (id);
ALTER TABLE employees ADD CONSTRAINT uc_name UNIQUE (name);

Running query in 'duckdb://'

RuntimeError: (duckdb.NotImplementedException) Not implemented Error: ALTER TABLE option not supported yet!
[SQL: ALTER TABLE employees ADD CONSTRAINT pk_id PRIMARY KEY (id);]
(Background on this error at: https://sqlalche.me/e/20/tw8g)
If you need help solving this issue, send us a message: https://ploomber.io/community


## Foreign Keys

This syntax doesn't work in DuckDB/JupySQL.

In [11]:
%%sql

CREATE TABLE departments (
    id INT PRIMARY KEY,
    name VARCHAR(100)
);
ALTER TABLE employees ADD CONSTRAINT fk_department_id FOREIGN KEY (department_id) REFERENCES departments(id);

Running query in 'duckdb://'

RuntimeError: (duckdb.NotImplementedException) Not implemented Error: ALTER TABLE option not supported yet!
[SQL: ALTER TABLE employees ADD CONSTRAINT fk_department_id FOREIGN KEY (department_id) REFERENCES departments(id);]
(Background on this error at: https://sqlalche.me/e/20/tw8g)
If you need help solving this issue, send us a message: https://ploomber.io/community


## Un-Indexing

In [12]:
%sql DROP INDEX idx_name;

Running query in 'duckdb://'

Success


## Dropping Constraints

This syntax doesn't work in DuckDB/JupySQL.

In [13]:
%sql ALTER TABLE employees DROP CONSTRAINT pk_id;

Running query in 'duckdb://'

RuntimeError: (duckdb.NotImplementedException) Not implemented Error: ALTER TABLE option not supported yet!
[SQL: ALTER TABLE employees DROP CONSTRAINT pk_id;]
(Background on this error at: https://sqlalche.me/e/20/tw8g)
If you need help solving this issue, send us a message: https://ploomber.io/community


# Understanding Primary Keys

In this code snippet, we demonstrate the usage of primary keys in SQL. 

First, we create a table called `employees` with an `id` column as the primary key. We insert some data into the table and attempt to insert a duplicate primary key value, which results in a primary key violation error.

Next, we show how to retrieve data from the table using a simple `SELECT` statement. We also demonstrate updating and deleting records using the primary key.

Then, we add a primary key constraint to an existing table using the `ALTER TABLE` statement. We attempt to insert a duplicate primary key value again, which results in an error. We then drop the primary key constraint and successfully insert the record with the previously duplicate primary key value.

Finally, we create another table called `products` with an auto-increment primary key column using the `SERIAL` data type. We insert data into the table without specifying the primary key value, and retrieve the data using a `SELECT` statement. We also attempt to insert a record with a specific primary key value, which results in an error due to the auto-increment property of the primary key column.

Note: The syntax and behavior of primary keys may vary slightly depending on the specific SQL database system being used.

## Base Table

In [2]:
%%sql

CREATE OR REPLACE TABLE employees (
    id INT PRIMARY KEY,
    name VARCHAR(50),
    age INT
);

INSERT INTO employees (id, name, age) VALUES (1, 'John Doe', 30);
INSERT INTO employees (id, name, age) VALUES (2, 'Jane Smith', 25);
INSERT INTO employees (id, name, age) VALUES (3, 'Mike Johnson', 35);

Running query in 'duckdb://'

Count


## Violating Uniqueness

Because a primary key is uniquely indexed, you can't insert a duplicate.

In [16]:
%sql INSERT INTO employees (id, name, age) VALUES (1, 'Duplicate', 40);

Running query in 'duckdb://'

RuntimeError: (duckdb.ConstraintException) Constraint Error: Duplicate key "id: 1" violates primary key constraint. If this is an unexpected constraint violation please double check with the known index limitations section in our documentation (docs - sql - indexes).
[SQL: INSERT INTO employees (id, name, age) VALUES (1, 'Duplicate' , 40);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
If you need help solving this issue, send us a message: https://ploomber.io/community


In [3]:
%sql SELECT * FROM employees;

Running query in 'duckdb://'

id,name,age
1,John Doe,30
2,Jane Smith,25
3,Mike Johnson,35


## Using Primary Key for Modifications

In [4]:
%%sql

UPDATE employees SET age = 31 WHERE id = 1;
DELETE FROM employees WHERE id = 2;
UPDATE employees SET age = 32 WHERE id = 4;
DELETE FROM employees WHERE id = 4;

Running query in 'duckdb://'

Count


## Auto-Increment Primary Key

There's no way to do this in DuckDB/JupySQL, but some systems have ways to auto-increment primary keys.

- SQLite automatically fills NULLs
- DuckDB (normally) has an AUTOINCREMENT keyword
- and many others

# Foreign Keys and Relations

In this code snippet, we demonstrate the usage of foreign keys and relations in SQL. 

First, we create two tables: "Customers" and "Orders". The "Customers" table has a primary key column `customer_id` and a `customer_name` column. The "Orders" table also has a primary key column `order_id`, an `order_date` column, and a `customer_id` column.

To establish a relationship between the two tables, we define a foreign key constraint on the `customer_id` column in the "Orders" table. The foreign key references the `customer_id` column in the "Customers" table, ensuring that only valid customer IDs can be inserted into the "Orders" table.

We then insert sample data into both tables to demonstrate the relationship. Two customers are inserted into the "Customers" table, and two orders are inserted into the "Orders" table, each associated with a specific customer.

Finally, we perform a query to retrieve the orders along with the corresponding customer information. The `JOIN` keyword is used to combine the "Orders" and "Customers" tables based on the matching `customer_id` values. The result includes the order ID, order date, and customer name.

Expected output:
```
order_id | order_date  | customer_name
--------------------------------------
1        | 2022-01-01 | John Doe
2        | 2022-01-02 | Jane Smith
```

Note: this example doesn't really show the benefits of having a foreign key since we did the equality check ourselves.  Here are some benefits:
- the constraint will prevent inserting invalid values in the table with the foreign key
- in some systems, deleting a customer will delete all of their orders automatically

In [2]:
%%sql

CREATE TABLE Customers (
    customer_id INT PRIMARY KEY,
    customer_name VARCHAR(50)
);

CREATE TABLE Orders (
    order_id INT PRIMARY KEY,
    order_date DATE,
    customer_id INT,
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);

INSERT INTO Customers (customer_id, customer_name)
VALUES (1, 'John Doe'),
       (2, 'Jane Smith');

INSERT INTO Orders (order_id, order_date, customer_id)
VALUES (1, '2022-01-01', 1),
       (2, '2022-01-02', 2);

Running query in 'duckdb://'

Count


In [3]:
%%sql

SELECT o.order_id, o.order_date, c.customer_name
FROM Orders o
JOIN Customers c ON o.customer_id = c.customer_id;

Running query in 'duckdb://'

order_id,order_date,customer_name
1,2022-01-01,John Doe
2,2022-01-02,Jane Smith


# How Indices Actually Work

1. You can think of a composite index as __appending the columns__ in the order specified in the index.
    -  eg. if you index on __(column1, column2, column3)__, then the indexed value is the concatenating of those 3 columns in that order
1. Internally, the index is basically a __binary search tree__, like a sorted map or set in Java.
    - it is O(logn) lookup, not O(1) (even for simple things like getting a single item by ID)
    - hash-based O(1) indexing is uncommon in relational databases (more common in NoSQL)
1. If an operation doesn't benefit from the indexing, then it just becomes a __sequential search__ (O(n))
1. An operation that searches or sorts can benefit from the indexing if you __use the columns in order__ with no other columns interveaning
    - eg. if the index has __(column1, column2, column3)__, then you can search or sort by column1+column2, column1+column2+column3, or just column1, but not column2+column3 or just column2 or column3.
1. This order applies across the whole query in certain places
    - eg. you might use column1 and column2 in the WHERE clause and column3 in the ORDER BY clause which comes later
1. It is ok to have a composite index that uses a subset of a __composite primary key__ (indexing is independent of primary keys)
1. You can have multiple indices combined in various ways to support the queries you need
   - but you might end up needing multiple denormalized tables or something if you can't arrange the phases efficiently
1. You can use the SQL `EXPLAIN` command to see how it queries the index for a given query